In [1]:
import random
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage
import sklearn
from sklearn.model_selection import train_test_split
import sktime
from sktime.datatypes._panel._convert import from_3d_numpy_to_nested
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report


In [2]:
def maximum(raw, box_size, mode='nearest'): # Maximum Filter
    raw_maximum=ndimage.maximum_filter(raw,box_size,mode=mode)
    return raw_maximum.real

def minimum(raw, box_size, mode='nearest'): # Maximum Filter
    raw_minimum=ndimage.minimum_filter(raw,box_size,mode=mode)
    return raw_minimum.real

def denoise_fft(data, ifftn): # Fast Fourier Transformation
    fft_signal = np.fft.fft(data)
    
    # Reconstruct the original signal
    fft_signal[ifftn:len(fft_signal)//2]=0
    fft_signal[len(fft_signal)//2:-ifftn]=0
    reconstructed_signal = np.fft.ifft(fft_signal)
    
    return reconstructed_signal.real

def smooth(x,beta): # Kaiser Window Smoothing
    window_len=11  # extending the data at beginning and at the end to apply the window at the borders

    s = np.r_[x[window_len-1:0:-1],x,x[-1:-window_len:-1]]
    w = np.kaiser(window_len,beta)
    y = np.convolve(w/w.sum(),s,mode='valid')
    return y[5:len(y)-5]


In [65]:

data_list = []
dtime_list = []

# for i in range(8):
#     dtime_list.append([0, f'data/paper/paper{i}.csv', f'data/new_rest/rest_time{i}.csv'])
#     dtime_list.append([0, f'data/rock/rock{i}.csv', f'data/new_rest/rest_time{i}.csv'])
#     dtime_list.append([1, f'data2/rock/rock{i}.csv', f'data2/new_rock/rock_time{i}.csv'])
#     dtime_list.append([2, f'data2/scissors/scissors{i}.csv', f'data2/new_scissors/scissors_time{i}.csv'])
#     dtime_list.append([3, f'data2/paper/paper{i}.csv', f'data2/new_paper/paper_time{i}.csv'])

# for i in range(6):
#     dtime_list.append([2, f'data2/aug_scissors/aug_scissors{i}.csv', f'data2/new_aug_scissors/aug_scissors_time{i}.csv'])

# for i in range(3,10):
#     dtime_list.append([1, f'data2/aug_rock/aug_rock{i}.csv', f'data2/new_aug_rock/aug_rock_time{i}.csv'])
#     dtime_list.append([2, f'data2/aug_scissors/aug_scissors{i}.csv', f'data2/new_aug_scissors/aug_scissors_time{i}.csv'])
#     dtime_list.append([3, f'data2/aug_paper/aug_paper{i}.csv', f'data2/new_aug_paper/aug_paper_time{i}.csv'])

# for i in range(6):
#     dtime_list.append([1, f'data3/rock/rock{i}.csv', f'data3/new_rock/rock_time{i}.csv'])
#     dtime_list.append([2, f'data3/scissors/scissors{i}.csv', f'data3/new_scissors/scissors_time{i}.csv'])
#     dtime_list.append([3, f'data3/paper/paper{i}.csv', f'data3/new_paper/paper_time{i}.csv'])

for i in range(1):
    dtime_list.append([0, f'data5/rest{i}.csv'])
    dtime_list.append([1, f'data5/rock{i}.csv'])
    dtime_list.append([5, f'data5/fire{i}.csv'])
    dtime_list.append([6, f'data5/paper{i}.csv'])

for i in range(4):
    dtime_list.append([0, f'data4/rest/rest{i}.csv'])
    dtime_list.append([1, f'data4/rock/rock{i}.csv'])
    dtime_list.append([2, f'data4/scissors/scissors{i}.csv'])
    dtime_list.append([3, f'data4/right/right{i}.csv'])
    dtime_list.append([4, f'data4/left/left{i}.csv'])
    dtime_list.append([5, f'data4/fire/fire{i}.csv'])
    dtime_list.append([6, f'data4/paper/paper{i}.csv'])
    dtime_list.append([7, f'data4/paper_left/paper_left{i}.csv'])
    dtime_list.append([8, f'data4/paper_right/paper_right{i}.csv'])
    # dtime_list.append([2, f'data4/bird_gun/bird_gun{i}.csv'])

for i in range(1):
    dtime_list.append([0, f'data5/rest{i}.csv'])
    dtime_list.append([1, f'data5/rock{i}.csv'])
    dtime_list.append([5, f'data5/fire{i}.csv'])
    dtime_list.append([6, f'data5/paper{i}.csv'])

for i in range(4):
    dtime_list.append([0, f'data4/rest/rest{i}.csv'])
    dtime_list.append([1, f'data4/rock/rock{i}.csv'])
    dtime_list.append([2, f'data4/scissors/scissors{i}.csv'])
    dtime_list.append([3, f'data4/right/right{i}.csv'])
    dtime_list.append([4, f'data4/left/left{i}.csv'])
    dtime_list.append([5, f'data4/fire/fire{i}.csv'])
    dtime_list.append([6, f'data4/paper/paper{i}.csv'])
    dtime_list.append([7, f'data4/paper_left/paper_left{i}.csv'])
    dtime_list.append([8, f'data4/paper_right/paper_right{i}.csv'])

print(dtime_list)

for num, data in dtime_list:
    print(num)
    print(data)
    df = pd.read_csv(data, header=None, names=['time', 'a0', 'a1', 'a2', 'a3', 'a4', 'a5'])


    df['time'] = pd.to_datetime(df['time'])

    asdf = []
    sec = 12
    while sec < 80:

        start_time = df['time'].iloc[0] + pd.Timedelta(seconds=sec)
        end_time = df['time'].iloc[0] + pd.Timedelta(seconds=sec+0.4)

        df_filtered = df[(df['time'] >= start_time) & (df['time'] <= end_time)]
        asdf.append([num, df_filtered])
        sec+=0.1

    for label, filtered_df in asdf:
        depths_list = []
        for i in range(6):
            column = f'a{i}'  # 필터링된 데이터의 각 채널 컬럼명
            depths = smooth(denoise_fft(maximum(filtered_df[column], 5)-minimum(filtered_df[column], 5), 5), 5)
            #diff = np.diff(depths)
            depths_list.append(depths)
            #depths_list.append(diff)
        data_list.append(np.append(label, depths_list))

df_2d = pd.DataFrame(data_list)

df_2d_cleaned = df_2d.dropna(axis=1, how='any')

[[0, 'data5/rest0.csv'], [1, 'data5/rock0.csv'], [5, 'data5/fire0.csv'], [6, 'data5/paper0.csv'], [0, 'data4/rest/rest0.csv'], [1, 'data4/rock/rock0.csv'], [2, 'data4/scissors/scissors0.csv'], [3, 'data4/right/right0.csv'], [4, 'data4/left/left0.csv'], [5, 'data4/fire/fire0.csv'], [6, 'data4/paper/paper0.csv'], [7, 'data4/paper_left/paper_left0.csv'], [8, 'data4/paper_right/paper_right0.csv'], [0, 'data4/rest/rest1.csv'], [1, 'data4/rock/rock1.csv'], [2, 'data4/scissors/scissors1.csv'], [3, 'data4/right/right1.csv'], [4, 'data4/left/left1.csv'], [5, 'data4/fire/fire1.csv'], [6, 'data4/paper/paper1.csv'], [7, 'data4/paper_left/paper_left1.csv'], [8, 'data4/paper_right/paper_right1.csv'], [0, 'data4/rest/rest2.csv'], [1, 'data4/rock/rock2.csv'], [2, 'data4/scissors/scissors2.csv'], [3, 'data4/right/right2.csv'], [4, 'data4/left/left2.csv'], [5, 'data4/fire/fire2.csv'], [6, 'data4/paper/paper2.csv'], [7, 'data4/paper_left/paper_left2.csv'], [8, 'data4/paper_right/paper_right2.csv'], [0, '

In [66]:
df_2d_cleaned.iloc[:, 1:].values

array([[ 1.39866714,  1.39989877,  1.40297837, ...,  1.18671439,
         1.20908755,  1.23203839],
       [ 1.33385386,  1.33926184,  1.35421558, ...,  1.98372525,
         1.9449366 ,  1.9000798 ],
       [ 1.48581734,  1.48987022,  1.50113144, ...,  1.79688054,
         1.77968192,  1.75883391],
       ...,
       [12.7060407 , 12.729648  , 12.79469652, ...,  3.15798908,
         3.05971186,  2.96635724],
       [13.92766145, 13.92275721, 13.90459081, ...,  6.65237179,
         6.64434467,  6.62788251],
       [14.19252029, 14.23603539, 14.3547279 , ...,  3.03647078,
         2.92022783,  2.8076304 ]])

In [67]:
df_2d_cleaned.iloc[:, 0].values

array([0., 0., 0., ..., 8., 8., 8.])

In [68]:
# 특징(X)와 라벨(y) 분리
X = df_2d_cleaned.iloc[:, 1:].values
y = df_2d_cleaned.iloc[:, 0].values


# 데이터셋 분할 (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [69]:
X_train.shape

(43584, 972)

In [70]:
y_train.shape

(43584,)

In [71]:
# CatBoost 모델 정의 및 학습
model = CatBoostClassifier(
    iterations=2000,        # 트리의 개수: 데이터 크기에 비해 너무 많지 않게 설정
    depth=6,                # 트리의 깊이: 과적합을 방지하기 위해 중간 정도로 설정
    learning_rate=0.02,      # 학습률: 일반적으로 0.1은 좋은 출발점
    loss_function='MultiClass',  # 다중 클래스 분류를 위한 손실 함수
    task_type='GPU',        # GPU 사용 설정
    devices='0',
    # l2_leaf_reg=3,          # L2 정규화 계수: 기본값으로 유지, 과적합 방지
    # border_count=254,       # 분리 기준의 개수: 기본값(254)보다 낮추어 모델 복잡도 조절
    # random_strength=10 ,      # 트리의 다양성을 증가시켜 과적합 방지
    # bagging_temperature=2,  # 샘플링 가중치: 과적합을 방지하기 위한 기본 설정
    # early_stopping_rounds=50,  # 성능 향상이 멈추면 조기 종료
    # random_seed=42,         # 결과 재현성을 위한 시드 설정
    verbose=100             # 학습 진행 상황을 출력하는 주기
)

model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 모델 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

0:	learn: 2.1183645	total: 71.4ms	remaining: 2m 22s
100:	learn: 0.4891963	total: 6.22s	remaining: 1m 57s
200:	learn: 0.2500850	total: 12.3s	remaining: 1m 49s
300:	learn: 0.1677245	total: 18.1s	remaining: 1m 42s
400:	learn: 0.1270764	total: 23.9s	remaining: 1m 35s
500:	learn: 0.1014313	total: 29.7s	remaining: 1m 28s
600:	learn: 0.0837088	total: 35.5s	remaining: 1m 22s
700:	learn: 0.0711990	total: 41.2s	remaining: 1m 16s
800:	learn: 0.0616241	total: 47s	remaining: 1m 10s
900:	learn: 0.0539173	total: 52.7s	remaining: 1m 4s
1000:	learn: 0.0477915	total: 58.4s	remaining: 58.3s
1100:	learn: 0.0430659	total: 1m 4s	remaining: 52.3s
1200:	learn: 0.0389997	total: 1m 9s	remaining: 46.4s
1300:	learn: 0.0356237	total: 1m 15s	remaining: 40.5s
1400:	learn: 0.0326096	total: 1m 20s	remaining: 34.6s
1500:	learn: 0.0299868	total: 1m 26s	remaining: 28.7s
1600:	learn: 0.0279048	total: 1m 32s	remaining: 22.9s
1700:	learn: 0.0260595	total: 1m 37s	remaining: 17.1s
1800:	learn: 0.0242393	total: 1m 43s	remainin

In [72]:
from joblib import dump, load

# 모델을 파일로 저장
dump(model, 'model777.joblib') 

# 777
#          0.0       1.00      1.00      1.00      1364
#          1.0       1.00      1.00      1.00      1336
#          2.0       0.99      0.99      0.99      1081
#          3.0       0.99      1.00      0.99      1077
#          4.0       0.99      0.99      0.99      1075
#          5.0       1.00      1.00      1.00      1344
#          6.0       0.99      0.99      0.99      1395
#          7.0       0.99      0.98      0.99      1099
#          8.0       1.00      1.00      1.00      1125

#     accuracy                           0.99     10896
#    macro avg       0.99      0.99      0.99     10896
# weighted avg       0.99      0.99      0.99     10896

# 113
#          0.0       1.00      1.00      1.00      1364
#          1.0       1.00      1.00      1.00      1336
#          2.0       0.98      0.98      0.98      1081
#          3.0       0.98      0.97      0.97      1077
#          4.0       0.99      1.00      0.99      1075
#          5.0       1.00      1.00      1.00      1344
#          6.0       0.99      1.00      0.99      1395
#          7.0       0.99      0.98      0.99      1099
#          8.0       1.00      1.00      1.00      1125

#     accuracy                           0.99     10896
#    macro avg       0.99      0.99      0.99     10896
# weighted avg       0.99      0.99      0.99     10896

# 333
#          0.0       1.00      1.00      1.00      1364
#          1.0       1.00      1.00      1.00      1336
#          2.0       0.98      0.97      0.97      1081
#          3.0       0.97      0.97      0.97      1077
#          4.0       0.99      0.99      0.99      1075
#          5.0       1.00      1.00      1.00      1344
#          6.0       0.99      0.99      0.99      1395
#          7.0       0.99      0.99      0.99      1099
#          8.0       1.00      1.00      1.00      1125

#     accuracy                           0.99     10896
#    macro avg       0.99      0.99      0.99     10896
# weighted avg       0.99      0.99      0.99     10896


['model777.joblib']

In [65]:
model.save_model('catboost_model.cbm', format="cbm")

In [38]:
data_list = []
time_list = []

df = pd.read_csv('data/rock/rock5.csv', header=None, names=['time', 'a0', 'a1', 'a2', 'a3', 'a4', 'a5'])
time_list = pd.read_csv('data/new_rock/rock_time5.csv', header=None)

df['time'] = pd.to_datetime(df['time'])

for i in range(len(time_list)):
    start_time = df['time'].iloc[0] + pd.Timedelta(seconds=time_list.iloc[i][0])
    end_time = df['time'].iloc[0] + pd.Timedelta(seconds=time_list.iloc[i][1])
    df_filtered = df[(df['time'] >= start_time) & (df['time'] <= end_time)]

    for i in range(6):
        column = f'a{i}'  # 필터링된 데이터의 각 채널 컬럼명
        depths = smooth(denoise_fft(maximum(df_filtered[column], 5)-minimum(df_filtered[column], 5), 5), 5)

    df_2d = pd.DataFrame(depths)

    df_2d_cleaned = df_2d.dropna(axis=1, how='any')

    result = df_2d_cleaned.to_numpy().flatten()

    print(np.round(model.predict_proba(result), 3))

[0.286 0.514 0.115 0.085]
[0.953 0.007 0.033 0.007]
[0.002 0.154 0.389 0.455]
[0.004 0.119 0.153 0.724]
[0.009 0.089 0.208 0.693]
[0.002 0.046 0.359 0.593]
[0.    0.318 0.323 0.358]
[0.031 0.082 0.523 0.365]
[0.015 0.094 0.296 0.595]
[0.597 0.235 0.133 0.035]
[0.157 0.225 0.382 0.236]
[0.024 0.098 0.309 0.569]
[0.97  0.013 0.014 0.004]
[0.96  0.006 0.023 0.01 ]
[0.143 0.107 0.388 0.363]
[0.952 0.007 0.02  0.02 ]
[0.949 0.012 0.029 0.01 ]
[0.041 0.095 0.645 0.219]
[0.065 0.279 0.424 0.232]
[0.015 0.098 0.299 0.587]
[0.165 0.108 0.198 0.529]
[0.956 0.007 0.028 0.009]
[0.574 0.295 0.09  0.04 ]
[0.001 0.656 0.136 0.208]
[0.148 0.065 0.137 0.651]
[0.02  0.416 0.256 0.307]
[0.036 0.071 0.449 0.444]
[0.207 0.264 0.332 0.197]


In [70]:
num_list = []
time_series = []

df = pd.read_csv('data4/scissors/scissors3.csv', header=None, names=['time', 'a0', 'a1', 'a2', 'a3', 'a4', 'a5'])

df['time'] = pd.to_datetime(df['time'])

sec = 15
while(True):
    if sec > 50:
        break
    
    start_time = df['time'].iloc[0] + pd.Timedelta(seconds=sec)
    end_time = df['time'].iloc[0] + pd.Timedelta(seconds=sec+0.4)
    df_filtered = df[(df['time'] >= start_time) & (df['time'] <= end_time)]
    print(df_filtered)

    depths_list = []
    for i in range(6):
        column = f'a{i}'  # 필터링된 데이터의 각 채널 컬럼명
        depths = smooth(denoise_fft(maximum(df_filtered[column], 5)-minimum(df_filtered[column], 5), 5), 5)
        depths_list.append(depths)
    df_2d = pd.DataFrame(depths_list)

    df_2d_cleaned = df_2d.dropna(axis=1, how='any')

    result = df_2d_cleaned.to_numpy().flatten()

    num_list.append([model.predict(result)[0], sec, sec+1, np.round(model.predict_proba(result), 3)])

    sec+=0.1

                        time   a0   a1   a2   a3   a4   a5
7054 2024-09-05 16:15:16.174  509  510  509  511  512  511
7055 2024-09-05 16:15:16.176  515  510  511  510  510  514
7056 2024-09-05 16:15:16.176  515  512  510  511  508  511
7057 2024-09-05 16:15:16.179  509  510  510  512  510  509
7058 2024-09-05 16:15:16.182  509  510  511  511  513  511
...                      ...  ...  ...  ...  ...  ...  ...
7238 2024-09-05 16:15:16.565  509  510  511  510  510  511
7239 2024-09-05 16:15:16.565  510  510  510  510  510  508
7240 2024-09-05 16:15:16.568  510  511  510  511  510  512
7241 2024-09-05 16:15:16.570  511  512  510  511  510  511
7242 2024-09-05 16:15:16.573  509  511  511  511  511  512

[189 rows x 7 columns]
                        time   a0   a1   a2   a3   a4   a5
7101 2024-09-05 16:15:16.274  510  509  509  509  509  510
7102 2024-09-05 16:15:16.274  510  511  511  511  509  512
7103 2024-09-05 16:15:16.276  510  511  510  510  509  511
7104 2024-09-05 16:15:16.280  51

In [71]:
import csv

with open('num_list.csv', mode='w', newline='') as file:
     writer = csv.writer(file)
     writer.writerows(num_list)